In [21]:
import torch.nn as nn
import torch.optim as optim
from mamba_ssm import Mamba
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [22]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
import torch

# 读取 Excel 数据
data = pd.read_excel("GMM based weather type.xlsx")  # 替换为你的数据文件路径
features = ['type', 'weather', 'wind', 'humidity', 'barometer', 'load']
data = data[features]

# 独热编码 'type' 列
encoder = OneHotEncoder(sparse_output=False)
type_encoded = encoder.fit_transform(data[['type']])
type_encoded_df = pd.DataFrame(type_encoded, columns=encoder.get_feature_names_out(['type']))

# MinMax 归一化其他列
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(data[['weather', 'wind', 'humidity', 'barometer', 'load']])
scaled_features_df = pd.DataFrame(scaled_features, columns=['weather', 'wind', 'humidity', 'barometer', 'load'])

# 合并独热编码和归一化结果
data_normalized = pd.concat([type_encoded_df, scaled_features_df], axis=1)

# 划分训练和测试集 (此处测试集为最后 48 条)
train_data = data_normalized.iloc[:-48].values
test_data = data_normalized.iloc[-48:].values

# 组序列函数
def prepare_sequences(data, sequence_length=24):
    X, y = [], []
    for i in range(len(data) - sequence_length):
        # X: 连续 24 小时的所有特征
        X.append(data[i:i + sequence_length])
        # y: 第 25 小时的 load (最后一列)
        y.append(data[i + sequence_length, -1])
    return np.array(X), np.array(y)

# 得到训练集
X_train, y_train = prepare_sequences(train_data, sequence_length=24)
# 得到测试集
X_test, y_test = prepare_sequences(test_data, sequence_length=24)

# 转化为 PyTorch Tensor
X_train = torch.tensor(X_train, dtype=torch.float32)  # [样本数, 24, 特征数]
y_train = torch.tensor(y_train, dtype=torch.float32)  # [样本数]
X_test = torch.tensor(X_test, dtype=torch.float32)    # [测试样本数, 24, 特征数]
y_test = torch.tensor(y_test, dtype=torch.float32)    # [测试样本数]

# 检查数据形状
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)


X_train shape: torch.Size([78816, 24, 14])
y_train shape: torch.Size([78816])
X_test shape: torch.Size([24, 24, 14])
y_test shape: torch.Size([24])


In [23]:
class MambaForecastModel(nn.Module):
    """
    使用 Mamba 模型进行预测：
    - d_model 对应输入维度 (input_size)
    - d_state 表示 SSM state 的扩展因子 (可视为隐藏维度)
    - d_conv 为卷积宽度
    - expand 为每个 Block 的扩张倍数
    """
    def __init__(self, input_size, d_state, d_conv, expand):
        super(MambaForecastModel, self).__init__()
        # Mamba 会输出与输入同维度的特征序列
        self.mamba = Mamba(
            d_model=input_size,  # 输入特征维度
            d_state=d_state,      # 状态扩展大小
            d_conv=d_conv,        # 局部卷积宽度
            expand=expand         # 块级扩张因子
        )
        # 将最后一个时间步的输出映射到单个值（预测 load）
        self.fc = nn.Linear(input_size, 1)

    def forward(self, x):
        """
        x shape: [batch_size, seq_len, input_size]
        Mamba 输出 shape 同 x，即 [batch_size, seq_len, input_size]
        这里我们只取最后一个时间步的向量，映射成 1 维输出。
        """
        out = self.mamba(x)             # [batch_size, seq_len, input_size]
        last_hidden = out[:, -1, :]     # [batch_size, input_size]
        out = self.fc(last_hidden)      # [batch_size, 1]
        return out.squeeze(-1)          # [batch_size], 与 y_train / y_test 对齐


In [24]:
# 实例化模型
# model = LSTMForecastModel(input_size=5, hidden_size=64, num_layers=1)
# 实例化 Mamba 模型
model = MambaForecastModel(
    input_size=14,   # 与数据特征维度相同
    d_state=16,     # 可视作“隐藏层”大小，可根据需求调整
    d_conv=4,
    expand=2
)
# 损失函数和优化器
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# 一般先把数据放到GPU(如果可用)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)
model.to(device)

X_train = X_train.to(device)
y_train = y_train.to(device)
X_test = X_test.to(device)
y_test = y_test.to(device)

# 超参
epochs = 200
batch_size = 32

# 开始训练
model.train()
for epoch in range(epochs):
    # 这里为了简单，写一个 mini-batch 的循环
    permutation = torch.randperm(X_train.size(0))
    total_loss = 0.0

    for i in range(0, X_train.size(0), batch_size):
        indices = permutation[i:i+batch_size]
        batch_x = X_train[indices]
        batch_y = y_train[indices]

        optimizer.zero_grad()
        outputs = model(batch_x)     # forward
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    if (epoch+1) % 10 == 0:
        avg_loss = total_loss / (X_train.size(0) // batch_size + 1)
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {avg_loss:.8f}")
        
torch.save(model.state_dict(), "GMM_clust_mamba_forecast_model_state.pth")

Using device: cuda
Epoch [10/200], Loss: 0.00023675
Epoch [20/200], Loss: 0.00016429
Epoch [30/200], Loss: 0.00014798
Epoch [40/200], Loss: 0.00013600
Epoch [50/200], Loss: 0.00012820
Epoch [60/200], Loss: 0.00011689
Epoch [70/200], Loss: 0.00010806
Epoch [80/200], Loss: 0.00010188
Epoch [90/200], Loss: 0.00009821
Epoch [100/200], Loss: 0.00009540
Epoch [110/200], Loss: 0.00009313
Epoch [120/200], Loss: 0.00009111
Epoch [130/200], Loss: 0.00009013
Epoch [140/200], Loss: 0.00008863
Epoch [150/200], Loss: 0.00008793
Epoch [160/200], Loss: 0.00008768
Epoch [170/200], Loss: 0.00008637
Epoch [180/200], Loss: 0.00008519
Epoch [190/200], Loss: 0.00008521
Epoch [200/200], Loss: 0.00008415


In [25]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = MambaForecastModel(
    input_size=14,   # 与数据特征维度相同
    d_state=16,     # 可视作“隐藏层”大小，可根据需求调整
    d_conv=4,
    expand=2
)
model.load_state_dict(torch.load("GMM_clust_mamba_forecast_model_state.pth", weights_only=True))
model.to(device)  # 确保模型移动到相应设备
model.eval()
# 确保输入张量也在正确的设备上
X_test = X_test.to(device)
y_test = y_test.to(device)  # 如果 y_test 需要在同一设备上，可选
with torch.no_grad():
    y_pred = model(X_test)  # [测试样本数]

# 转回CPU的numpy，方便用sklearn度量
y_test_cpu = y_test.cpu().numpy()
y_pred_cpu = y_pred.cpu().numpy()

#============================#
#      进行“反归一化”      #
#============================#

# 1) 准备一个与原特征数相同（5列）的零矩阵
test_data_temp = np.zeros((y_test_cpu.shape[0], 5))
pred_data_temp = np.zeros((y_pred_cpu.shape[0], 5))

# 2) 仅将最后一列（对应 load ）替换为测试值和预测值
test_data_temp[:, -1] = y_test_cpu
pred_data_temp[:, -1] = y_pred_cpu

# 3) 使用之前的 scaler 做逆变换
test_data_unscaled = scaler.inverse_transform(test_data_temp)
pred_data_unscaled = scaler.inverse_transform(pred_data_temp)

# 4) 取出反归一化后的 load 列
y_test_unscaled = test_data_unscaled[:, -1]
y_pred_unscaled = pred_data_unscaled[:, -1]

#============================#
#   计算原始尺度下的误差     #
#============================#

# 在原始量纲下计算 MSE, MAE, R^2, MAPE
mse = mean_squared_error(y_test_unscaled, y_pred_unscaled)
mae = mean_absolute_error(y_test_unscaled, y_pred_unscaled)
r2 = r2_score(y_test_unscaled, y_pred_unscaled)
mape = np.mean(np.abs((y_test_unscaled - y_pred_unscaled) / (y_test_unscaled + 1e-8))) * 100

print("Test MSE (Unscaled):  ", mse)
print("Test MAE (Unscaled):  ", mae)
print("Test R^2 (Unscaled):  ", r2)
print("Test MAPE (Unscaled): {:.2f}%".format(mape))


Test MSE (Unscaled):   2140.1165942499315
Test MAE (Unscaled):   38.18388245595494
Test R^2 (Unscaled):   0.9858527901528553
Test MAPE (Unscaled): 1.27%


In [26]:
# 初始化预测值存储
y_pred_unscaled = []  # 用于存储滚动预测的反归一化结果

# 初始化第一个测试序列，并将其移动到同一设备
current_sequence = X_test[0].unsqueeze(0).clone().to(device)  # [1, 24, 14]
last_pre = current_sequence[0, -1, 4].item()  # 初始化为第一个序列的最后一个 load 值

with torch.no_grad():
    for i in range(len(X_test)):
        # 将当前序列克隆并移动到设备
        current_sequence = X_test[i].unsqueeze(0).clone().to(device)

        # 将预测值替换到当前序列的最后一个时间步的 load
        current_sequence[0, -1, 4] = torch.tensor(last_pre, dtype=torch.float32, device=device)

        # 模型预测当前时间步的 load
        pred = model(current_sequence)  # [1, 1]
        y_pred_unscaled.append(pred.item())  # 记录预测值

        # 更新 last_pre，作为下一步的输入
        last_pre = pred.item()

# 转换为 NumPy 数组
y_pred_unscaled = np.array(y_pred_unscaled)

#============================#
#      进行“反归一化”      #
#============================#

# 1) 准备一个与原特征数相同（14列）的零矩阵
test_data_temp = np.zeros((y_test.shape[0], 5))
pred_data_temp = np.zeros((len(y_pred_unscaled), 5))

# 2) 仅将第 14 列（对应 load ）替换为真实值和预测值
test_data_temp[:, -1] = y_test.cpu().numpy()
pred_data_temp[:, -1] = y_pred_unscaled

# 3) 使用之前的 scaler 做逆变换
test_data_unscaled = scaler.inverse_transform(test_data_temp)
pred_data_unscaled = scaler.inverse_transform(pred_data_temp)

# 4) 取出反归一化后的 load 列
y_test_unscaled = test_data_unscaled[:, -1]
y_pred_unscaled = pred_data_unscaled[:, -1]

#============================#
#   计算原始尺度下的误差     #
#============================#

# 在原始量纲下计算 MSE, MAE, R^2, MAPE
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

mse = mean_squared_error(y_test_unscaled, y_pred_unscaled)
mae = mean_absolute_error(y_test_unscaled, y_pred_unscaled)
r2 = r2_score(y_test_unscaled, y_pred_unscaled)
mape = np.mean(np.abs((y_test_unscaled - y_pred_unscaled) / (y_test_unscaled + 1e-8))) * 100

print("Test MSE (Unscaled):  ", mse)
print("Test MAE (Unscaled):  ", mae)
print("Test R^2 (Unscaled):  ", r2)
print("Test MAPE (Unscaled): {:.2f}%".format(mape))

Test MSE (Unscaled):   11447.19757284309
Test MAE (Unscaled):   90.489140070652
Test R^2 (Unscaled):   0.9243284657201142
Test MAPE (Unscaled): 3.11%
